# G2Engine Guide - Adding records

The `addRecord()` method creates a JSON string with information about your Senzing version.

More information:

* [G2Engine Reference](senzing-G2Engine-reference.ipynb)

## Prepare Environment

In [ ]:
import com.senzing.g2.engine.G2Engine;
import com.senzing.g2.engine.G2JNI;
import com.senzing.g2.engine.Result;

### Helper class for Json Rendering

In [ ]:
%%loadFromPOM
<dependency>
    <groupId>org.glassfish</groupId>
    <artifactId>javax.json</artifactId>
    <version>1.1.4</version>
</dependency>

In [ ]:
import javax.json.*;
import static java.util.Collections.*;
import static javax.json.stream.JsonGenerator.PRETTY_PRINTING;

In [ ]:
public class JsonUtil {
    private static final JsonWriterFactory PRETTY_FACTORY
        = Json.createWriterFactory(singletonMap(PRETTY_PRINTING, true));
        
    private static final JsonWriterFactory UGLY_FACTORY
        = Json.createWriterFactory(emptyMap());
    
    public static String toJsonText(JsonValue val) {
        return toJsonText(val, true);
    }

    public static String toJsonText(JsonValue val, boolean prettyPrint) {
        JsonWriterFactory factory = (prettyPrint) ? PRETTY_FACTORY : UGLY_FACTORY;
        StringWriter sw = new StringWriter();
        JsonWriter writer = factory.createWriter(sw);
        writer.write(val);
        sw.flush();
        return sw.toString();
    }
    
    public static JsonObject parseJsonObject(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readObject();
  }

    public static JsonArray parseJsonArray(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readArray();
  }
}

In [ ]:
import java.util.UUID;
String str;
public static void RenderJSON(Object obj){
    str = obj.toString();
    JsonObject json = JsonUtil.parseJsonObject(str);
    String Config = JsonUtil.toJsonText(json, false);
    UUID id = UUID.randomUUID();
    String uuid = id.toString();
    String div = "<div id=\""+ uuid +"\" style=\"height:100%; width:100%; background-color: LightCyan\"></div>";
    display(div, "text/html");
    String jav = "require([\"https://rawgit.com/caldwell/renderjson/master/renderjson.js\"], function() {document.getElementById(\'"+ uuid +"\').appendChild(renderjson("+json+"))});";
    display(jav, "application/javascript");
}

### Initialize Senzing configuration

Using environment variables and default values, create `senzingConfigJson`.
This value is used when instantiating Senzing objects.

In [ ]:
// Get variables used in constructing Senzing Engine configuration.
String senzingConfigJson=System.getenv("SENZING_ENGINE_CONFIGURATION_JSON");

String configPath = System.getenv("SENZING_ETC_DIR");
if (configPath == null) {
    configPath = "/etc/opt/senzing";
}

String supportPath = System.getenv("SENZING_DATA_VERSION_DIR");
if (supportPath == null) {
    supportPath = "/opt/senzing/data";
}

String g2Path = System.getenv("SENZING_G2_DIR");
if (g2Path == null) {
    g2Path = "/opt/senzing/g2";
}

String resourcePath = g2Path + "/resources";

String sqlConnection = System.getenv("SENZING_SQL_CONNECTION");
if (sqlConnection == null) {
    sqlConnection = "sqlite3://na:na@/var/opt/senzing/sqlite/G2C.db";
}

    // Construct the JSON string used for Senzing Engine configuration.
if(senzingConfigJson==null)
    senzingConfigJson = "{"
       + "\"PIPELINE\": {"
       +     "\"CONFIGPATH\": \"" + configPath + "\","
       +     "\"SUPPORTPATH\": \"" + supportPath + "\","
       +     "\"RESOURCEPATH\": \"" + resourcePath + "\""
       + "},"
       + "\"SQL\": {"
       +     "\"CONNECTION\": \"" + sqlConnection + "\""
       + "}}";

RenderJSON(senzingConfigJson);

## G2Engine

### G2Engine initialization

To start using Senzing G2Engine, create and initialize an instance.
This should be done once per process.
The `initV2()` method accepts the following parameters:

- **moduleName:** A short name given to this instance of the G2Engine
  object.
- **senzingConfigJson:** A JSON string containing configuration parameters.
- **verboseLogging:** A boolean which enables diagnostic logging.
- **configID:** (optional) The identifier value for the engine configuration
  can be returned here.

Calling this function will return "0" upon success.

In [ ]:
G2Engine g2engine = new G2JNI();

String moduleName = "ExampleG2Engine";
boolean verboseLogging = true;
int return_code = g2engine.initV2(moduleName, senzingConfigJson, verboseLogging);
System.out.print(return_code);

### addRecord

Once the Senzing engine is initialized, use addRecord() to load a record into the Senzing repository -- addRecord() can be called as many times as desired and from multiple threads at the same time. The addRecord() function returns "0" upon success, and accepts four parameters as input:

- **dataSourceCode:** The name of the data source the record is associated with. This value is configurable to the system
- **recordID:** The record ID, used to identify distinct records
- **jsonData:** A JSON document with the attribute data for the record
- **loadID:** The observation load ID for the record; value can be null and will default to data_source


In [ ]:
String dataSourceCode = "TEST";
String recordID = "1";
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"JANE\", \"NAME_LAST\": \"SMITH\", \"ADDR_TYPE\": \"HOME\", \"ADDR_LINE1\": \"653 STATE ROUTE 7\", \"ADDR_CITY\": \"FRESNO\", \"ADDR_STATE\": \"CA\", \"ADDR_POSTAL_CODE\": \"55073-1234\"}";
String loadID = null;

/** addRecord */
int return_code= g2engine.addRecord(dataSourceCode, recordID, jsonData, loadID);

/** addRecordWithReturnedRecordID */
StringBuffer recordID2 = new StringBuffer();
int ret2 = g2engine.addRecordWithReturnedRecordID(dataSourceCode, recordID2, jsonData, loadID);

System.out.print("New Record ID: " + recordID2.toString());

/** addRecordWithInfo */
StringBuffer responseBuffer = new StringBuffer();
int flags = 0;
int ret3 = g2engine.addRecordWithInfo(dataSourceCode, recordID, jsonData, loadID, flags, responseBuffer);

RenderJSON(responseBuffer);

### getRecordV2

Use `getRecord()` to retrieve a single record from the data repository; the record is assigned in JSON form to a user-designated buffer, and the function itself returns "0" upon success. Once the Senzing engine is initialized, `getRecord()` can be called as many times as desired and from multiple threads at the same time. The `getRecord()` function accepts the following parameters as input:

- **dataSourceCode:** The name of the data source the record is associated with. This value is configurable to the system
- **recordID:** The record ID, used to identify the record for retrieval
- **flags:** Control flags for specifying what data about the
  entity to retrieve.
- **response:** A memory buffer for returning the response document; if an error occurred, an error response is stored here

In [ ]:
int flags = 0;
String dataSourceCode = "TEST";
String recordID = "1";
StringBuffer response = new StringBuffer();
int return_code= g2engine.getRecordV2(dataSourceCode, recordID, flags, response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response)

## Search

### getEntityByRecordIDV2

Use `getEntityByRecordIDV2()` to retrieve entity data based on the record ID of a particular data record. This function accepts the following parameters as input:

- **dataSourceCode:** The name of the data source the record is associated with. This value is configurable to the system
- **recordID:** The record ID for a particular data record
- **flags:** Control flags for specifying what data about the
  entity to retrieve.
- **response:** A memory buffer for returning the response document; if an error occurred, an error response is stored here.

In [ ]:
String dataSourceCode = "TEST";
String recordID = "1";

StringBuffer response = new StringBuffer();

int return_code = g2engine.getEntityByRecordIDV2(dataSourceCode, recordID, flags, response);
JsonObject jsonObject = JsonUtil.parseJsonObject(response.toString());
long entityID = jsonObject.getJsonObject("RESOLVED_ENTITY").getJsonNumber("ENTITY_ID").longValue();
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

### getEntityByEntityIDV2

Entity searching is a key component for interactive use of Entity Resolution
intelligence.
The core Senzing engine provides real-time search capabilities that are
easily accessed via the Senzing API.
Senzing offers methods for entity searching, all of which can be called as
many times
as desired and from multiple threads at the same time (and all of which
return "0" upon success).

Use `getEntityByEntityIDV2()` to retrieve entity data based on the ID of a
resolved identity.
This function accepts the following parameters as input:

- **entityID:** The numeric ID of a resolved entity
- **flags:** Control flags for specifying what data about the
  entity to retrieve.
- **response:** A memory buffer for returning the response
  document; if an error occurred, an error response is stored here.

In [ ]:
response = new StringBuffer();

int ret2 = g2engine.getEntityByEntityIDV2(entityID, flags, response);
RenderJSON(response);

### searchByAttributes

Use `searchByAttributesV2()` to retrieve entity data based on a user-specified set of entity attributes. This function accepts the following parameters as input:

- **jsonData:** A JSON document with the attribute data to search for
- **flags:** Control flags for specifying what data about the
  entity to retrieve.
- **response:** A memory buffer for returning the response document; if an error occurred, an error response is stored here.

In [ ]:
String dataSourceCode = "TEST";
String recordID = "1";
int flags = 0;
response = new StringBuffer();

int return_code = g2engine.searchByAttributesV2(jsonData, flags, response);
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

## Cleanup

### Purge Repository
To purge the G2 repository, use the aptly named `purgeRepository()` method. This will remove every record in your current repository.

In [ ]:
int return_code= g2engine.purgeRepository();
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print(return_code)

### destroy
Once all searching is done in a process call `destroy()` to uninitialize Senzing and clean up resources. You should always do this once at the end of each process.

In [ ]:
int return_code = g2engine.destroy();

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print(return_code);